In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
import yaml
import os
import glob

import sys
sys.path.append('../')
from utils import *

In [ ]:
with open("../config.yaml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

for section in cfg:
    RANDOM_STATE_SEED = cfg["variables"]["RANDOM_STATE_SEED"]
    THRESHOLD = cfg["variables"]["THRESHOLD"]
    THRESHOLD_LOSS = cfg["variables"]["THRESHOLD_LOSS"]

    PROCESSED_DATA_DIR = cfg["data_path"]["PROCESSED_DATA_DIR"]

DATA_DIR = 'D:/code/gr1/data/unraveled/data/network_flows'

In [ ]:
dataset_paths = [os.path.join(DATA_DIR,  dataset_path) for dataset_path in os.listdir(DATA_DIR)]
all_csv_files = []
for dataset_path in dataset_paths:
    csv_files_directory = glob.glob(dataset_path + '/*.csv')
    all_csv_files.extend(csv_files_directory)

# ## Load the data

In [ ]:
import pandas as pd

df = pd.read_csv('D:/code/gr1/data/dataset/unraveled_defender_response.csv')

In [ ]:
unique_stages = df['Stage'].unique()
print(f"Các giá trị duy nhất trong cột 'Stage': {unique_stages}")

In [ ]:
list_features = [
    'bidirectional_duration_ms', 'src2dst_duration_ms',
    'dst2src_duration_ms', 'src2dst_packets', 'dst2src_packets',
    'src2dst_bytes', 'dst2src_bytes', 'src2dst_max_ps', 'src2dst_min_ps',
    'src2dst_mean_ps', 'src2dst_stddev_ps', 'dst2src_max_ps', 'dst2src_min_ps',
    'dst2src_mean_ps', 'dst2src_stddev_ps', 'bidirectional_mean_piat_ms',
    'bidirectional_stddev_piat_ms', 'bidirectional_max_piat_ms', 
    'bidirectional_min_piat_ms','src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms', 
    'src2dst_max_piat_ms', 'src2dst_min_piat_ms', 'dst2src_mean_piat_ms',
    'dst2src_stddev_piat_ms', 'dst2src_max_piat_ms', 'dst2src_min_piat_ms', 
    'bidirectional_fin_packets', 'src2dst_fin_packets', 'dst2src_fin_packets', 
    'bidirectional_syn_packets', 'src2dst_syn_packets', 'dst2src_syn_packets',
    'bidirectional_rst_packets', 'src2dst_rst_packets', 'dst2src_rst_packets',
    'bidirectional_psh_packets', 'src2dst_psh_packets', 'dst2src_psh_packets', 
    'bidirectional_ack_packets', 'src2dst_ack_packets', 'dst2src_ack_packets', 
    'bidirectional_urg_packets', 'src2dst_urg_packets', 
    'bidirectional_cwr_packets', 'src2dst_cwr_packets', 
    'bidirectional_ece_packets', 'src2dst_ece_packets', 'Stage'
]

df = df[list_features]

In [ ]:
import pandas as pd

# Giả sử stage_mapping là từ điển mà bạn đã định nghĩa
stage_mapping = {
    'Benign': 0,        
    'Reconnaissance': 1,     
    'Establish Foothold': 2,
    'Lateral Movement': 3,
    'Data Exfiltration': 4,
    'Cover up': 5
}

print(df.columns)

# Giả sử df là DataFrame đã đọc từ một tập tin CSV và có cột 'Stage'
# In ra các giá trị duy nhất trong cột 'Stage'
unique_stages = df['Stage'].unique()
print(f"Các giá trị duy nhất trong cột 'Stage': {unique_stages}")


In [ ]:
import matplotlib.pyplot as plt
# Đếm số lượng mẫu cho từng nhãn
stage_counts = df['Stage'].value_counts().sort_index()

# Vẽ biểu đồ
plt.figure(figsize=(10, 6))
plt.bar(stage_counts.index, stage_counts.values, color='skyblue', alpha=0.8)

# Thêm nhãn, tiêu đề và lưới
plt.title('Distribution of Stages', fontsize=16)
plt.xlabel('Stage', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(ticks=stage_counts.index, labels=[
    'Benign',
    'Reconnaissance',
    'Establish Foothold',
    'Lateral Movement',
    'Data Exfiltration',
    'Cover up'
], rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', linewidth=0.7, alpha=0.7)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.utils import resample
import pandas as pd

# Định nghĩa số lượng mẫu mong muốn cho từng nhãn
desired_samples = {
    0: 500,
    1: 3000,
    2: 3000,
    3: 1000,
    4: 1000,
    5: 359
}

# Chia dữ liệu thành từng nhãn và resample theo số lượng mong muốn
df_balanced = pd.concat([
    resample(
        df[df['Stage'] == stage],
        replace=False,  # Không thay thế
        n_samples=desired_samples[stage],  # Số lượng mẫu mong muốn
        random_state=42
    )
    for stage in desired_samples.keys()
])

# Kiểm tra kết quả
print(df_balanced['Stage'].value_counts())


In [ ]:
from sklearn.utils import resample
import pandas as pd
# Tăng số lượng nhãn 5 lên (oversampling)
desired_samples_for_label_5 = 1000  # Số lượng mong muốn cho nhãn 5
df_label_5 = df_balanced[df_balanced['Stage'] == 5]  # Lấy các mẫu của nhãn 5

df_label_5_oversampled = resample(
    df_label_5,
    replace=True,  # Cho phép nhân bản
    n_samples=desired_samples_for_label_5,  # Tăng lên số lượng mong muốn
    random_state=42
)

# Kết hợp lại toàn bộ dữ liệu
df_balanced_final = pd.concat([
    df_balanced[df_balanced['Stage'] != 5],  # Các nhãn khác
    df_label_5_oversampled  # Nhãn 5 đã oversample
])

# Kiểm tra kết quả cuối cùng
print(df_balanced_final['Stage'].value_counts())

### Split the data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

X = df_balanced.drop('Stage', axis=1)
y = df_balanced['Stage']
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

In [ ]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
xgb_apt2021 = xgb.XGBClassifier(random_state = 42)

In [ ]:
xgb_apt2021.fit(x_train, y_train)


In [ ]:
y_pred = xgb_apt2021.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

print(classification_report(y_test, y_pred))